In [ ]:
# Import Python package
from binhosupernova import getConnectedSupernovaDevicesList
from binhosupernova.supernova import Supernova
from binhosupernova.commands.i3c.definitions import *

#### 1. List all the Supernova devices conneted to the PC host

In [2]:
getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&11c48e71&2&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '39EB517A3B54BE5E95DBDDCA22B6A16',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create device instance
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## I3C Protocol API

The I3C Protocol API methods are described below.

### 1. Configure the Supernova as an I3C Controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [9]:
supernova.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_250_KHZ, I2cTransferRate._400KHz)

{'module': 1, 'opcode': 0, 'message': 'I3C CONTROLLER INIT request success'}

>> New message from SUPERNOVA:
{'id': 2, 'command': 'I3C CONTROLLER INIT', 'result': 'SUCCESS'}


### 2. Set I3C voltage

``Supernova.setI3cVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the I3C bus in mV, ranging from 800 mV up to 3300 mV. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [800, 3300].

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [10]:
request_result = supernova.setI3cVoltage(getId(), 3300)

>> New message from SUPERNOVA:
{'id': 3, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


### 3. I3C bus initialization

There are two options:
1. Initialize the bus with an **empty target devices table** and scan the I3C bus.
2. Add target to the target device table before initializing the bus.

In this case, the I3C Controller performs an RSTDAA and ENTDAA CCC. If one or more I3C Targets acknowledge the Broadcast Address 0x7E, then the Supernova 
will assign the lower available and valid dynamic address.

In [12]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 4, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


Add targets to the target devices table using a Python dictionary

In [13]:
ICM_STATIC_ADDRESS = 0x69
ICM_DYNAMIC_ADDRESS = 0x08

ICM_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.DO_NOT_USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

table = {0: {'staticAddress': ICM_STATIC_ADDRESS,
        'dynamicAddress': ICM_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'configuration': ICM_CONFIGURATION}}

request_result = supernova.i3cControllerInitBus(getId(), table)

>> New message from SUPERNOVA:
{'id': 5, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [14]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 6, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 1, 'table': [{'static_address': 105, 'dynamic_address': 8, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}]}


### 4. I3C Transfers

#### 4.1 Update PID, BCR and DCR

In [15]:
TARGET_ADDR = ICM_DYNAMIC_ADDRESS

request_result = supernova.i3cGETPID(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 7, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETPID', 'payload_length': 6, 'payload': [4, 106, 0, 0, 0, 0]}


In [16]:
TARGET_ADDR = ICM_DYNAMIC_ADDRESS

request_result = supernova.i3cGETBCR(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 8, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETBCR', 'payload_length': 1, 'payload': [39]}


In [17]:
TARGET_ADDR = ICM_DYNAMIC_ADDRESS

request_result = supernova.i3cGETDCR(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 9, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETDCR', 'payload_length': 1, 'payload': [160]}


#### 4.2 Set MWL and MRL

In [18]:
TARGET_ADDR = ICM_DYNAMIC_ADDRESS
MAXIMUM_READ_LEN = 512

request_result = supernova.i3cDirectSETMRL(getId(), TARGET_ADDR, MAXIMUM_READ_LEN)

>> New message from SUPERNOVA:
{'id': 10, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMRL', 'payload_length': 2}


In [19]:
MAXIMUM_WRITE_LEN = 256

request_result = supernova.i3cBroadcastSETMWL(getId(), MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{'id': 11, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETMWL', 'payload_length': 3}


#### 4.3 Disable In-Band interrupts

In [20]:
request_result = supernova.i3cDirectDISEC(getId(),ICM_DYNAMIC_ADDRESS,[DISEC.DISHJ,DISEC.DISINT])

>> New message from SUPERNOVA:
{'id': 12, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_DISEC', 'payload_length': 1}


#### 4.4 Configure sensor

1. Write ``REG_BANK_SEL``

Change register bank to bank 0.

In [22]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x00

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [REG_BANK_SEL], [VALUE])

>> New message from SUPERNOVA:
{'id': 13, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


2. Write ``PWR_MGMT0`` register

Turn off Accelerometer to modify registers value.

In [23]:
# PWR_MGMT0 register address
PWR_MGMT0   = 0x4E
VALUE       = 0x20

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[PWR_MGMT0],[VALUE])

>> New message from SUPERNOVA:
{'id': 14, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


3. Write ``DRIVE_CONFIG`` register

Set ``I2C_SLEW_RATE`` and ``SPI_SLEW_RATE``

In [24]:
# DRIVE_CONFIG register address
DRIVE_CONFIG    = 0x13
VALUE           = 0x05

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [DRIVE_CONFIG], [VALUE])

>> New message from SUPERNOVA:
{'id': 15, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


4. Write ``FIFO_CONFIG`` register

Set Fifo mode: Stream-to-FIFO Mode

In [25]:
# FIFO_CONFIG register address
FIFO_CONFIG = 0x16
VALUE       = 0x40

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [FIFO_CONFIG], [VALUE])

>> New message from SUPERNOVA:


{'id': 16, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


5. Write ``FIFO_CONFIG1`` register

- FIFO read can be partial, and resume from last read point
- Trigger FIFO watermark interrupt on every ODR (DMA write) if FIFO_COUNT ≥ FIFO_WM_TH
- Enable accelerometer packets to go to FIFO

In [26]:
# FIFO_CONFIG1 register address
FIFO_CONFIG1    = 0x5F
VALUE           = 0x61

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [FIFO_CONFIG1], [VALUE])

>> New message from SUPERNOVA:
{'id': 17, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


In [27]:
# FIFO_CONFIG2 register address
FIFO_CONFIG2   = 0x60
VALUE          = [0x0F, 0x00]

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [FIFO_CONFIG2], VALUE)

>> New message from SUPERNOVA:
{'id': 18, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 2}


7. Write ``ACCEL_CONFIG0``

Set Accelerometer ODR selection for UI interface output to 1.5625Hz (LP mode).

In [28]:
# ACCEL_CONFIG0 register address
ACCEL_CONFIG0   = 0x50
#VALUE           = 0x0A # 25Hz (LP or LN mode)
VALUE           = 0x0E # 1.5625Hz (LP mode)

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(), ICM_DYNAMIC_ADDRESS, TransferMode.I3C_SDR, [ACCEL_CONFIG0], [VALUE])

>> New message from SUPERNOVA:
{'id': 19, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


8. Write ``REG_BANK_SEL``

Change register bank to bank 1.

In [29]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x01

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[REG_BANK_SEL],[VALUE])

>> New message from SUPERNOVA:
{'id': 20, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


9. Write ``SENSOR_CONFIG0``

Enable accelerometer X, Y, and Z data.

In [30]:
# SENSOR_CONFIG0 register address
SENSOR_CONFIG0  = 0x03
VALUE           = 0x38

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[SENSOR_CONFIG0],[VALUE])

>> New message from SUPERNOVA:
{'id': 21, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


10. Write ``INTF_CONFIG4``

Set I3C_BUS_MODE

In [31]:
# INTF_CONFIG4 register address
INTF_CONFIG4    = 0x7A
VALUE           = 0x02

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[INTF_CONFIG4],[VALUE])

>> New message from SUPERNOVA:
{'id': 22, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


11. Write ``INTF_CONFIG6``

Enable I3C In-Band Interrupt

In [32]:
# INTF_CONFIG6 register address
INTF_CONFIG6    = 0x7C
VALUE           = 0x1F

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[INTF_CONFIG6],[VALUE])

>> New message from SUPERNOVA:
{'id': 23, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


12. Write ``REG_BANK_SEL``

Change register bank to bank 4.

In [33]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x04

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[REG_BANK_SEL],[VALUE])

>> New message from SUPERNOVA:
{'id': 24, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


13. Write ``INT_SOURCE8``

Route FIFO threshold interrupt to I3C In-Band Interrupt

In [34]:
# INT_SOURCE8 register address
INT_SOURCE8 = 0x4F
VALUE       = 0x04

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[INT_SOURCE8],[VALUE])

>> New message from SUPERNOVA:
{'id': 25, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


14. Write ``REG_BANK_SEL``

Change register bank to bank 0.

In [35]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x00

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[REG_BANK_SEL],[VALUE])

>> New message from SUPERNOVA:
{'id': 26, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


15. Write ``PWR_MGMT0`` register

Enable Temperature sensor and Accelerometer in Low Power mode

In [36]:
# PWR_MGMT0 register address
PWR_MGMT0   = 0x4E
#VALUE       = 0x03 # 11: Places accelerometer in Low Noise (LN) Mode
VALUE       = 0x02 # 10: Places accelerometer in Low Power (LP) Mode

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),ICM_DYNAMIC_ADDRESS,TransferMode.I3C_SDR,[PWR_MGMT0],[VALUE])

>> New message from SUPERNOVA:
{'id': 27, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 1}


#### 4.5 Enable In-Band interrupts

In [37]:
request_result = supernova.i3cDirectENEC(getId(),ICM_DYNAMIC_ADDRESS,[ENEC.ENINT])

>> New message from SUPERNOVA:
{'id': 28, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_ENEC', 'payload_length': 1}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}


>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id

In [40]:
request_result = supernova.i3cDirectDISEC(getId(),ICM_DYNAMIC_ADDRESS,[DISEC.DISHJ,DISEC.DISINT])

>> New message from SUPERNOVA:
{'id': 31, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_DISEC', 'payload_length': 1}


In [39]:
request_result = supernova.i3cDirectENEC(getId(),ICM_DYNAMIC_ADDRESS,[ENEC.ENINT])

>> New message from SUPERNOVA:
{'id': 30, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_ENEC', 'payload_length': 1}


>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id

In [41]:
request_result = supernova.i3cBroadcastENEC(getId(),[ENEC.ENINT])

>> New message from SUPERNOVA:
{'id': 32, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_ENEC', 'payload_length': 2}


>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id

In [42]:
request_result = supernova.i3cBroadcastDISEC(getId(),[DISEC.DISINT])

>> New message from SUPERNOVA:
{'id': 0, 'command': 'I3C CONTROLLER IBI REQUEST NOTIFICATION', 'address': 8, 'result': 'IBI_REQUEST_ACCEPTED_WITH_PAYLOAD', 'payload_length': 1, 'payload': [2]}
>> New message from SUPERNOVA:
{'id': 33, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_DISEC', 'payload_length': 2}


### 5. Reset I3C Bus

`Supernova.i3cControllerResetBus(id)`

This method requests the Supernova to: 1. issue an RSTDAA CCC, 2. free the already assigned dynamic addresses and 3. clean up the target devices table. 

- `id: int`: A 2-byte integer that represents the transfer ID.

In [43]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 34, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


### 6. Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [44]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}